In [1]:
import pickle
import pandas as pd
import numpy as np

from astropy.io import ascii
from astropy.coordinates import SkyCoord
from SPLASH.pipeline import Splash_Pipeline

# Load in data

In [2]:
# Load in bts data
bts_hosts = ascii.read('/Users/adamboesky/Research/ay98/clean_data/bts_hosts.ecsv')
bts_df = bts_hosts.to_pandas()
bts_df.head()

,objID_3pi,raStack_3pi,decStack_3pi,primaryDetection_3pi,gKronMag_3pi,rKronMag_3pi,iKronMag_3pi,zKronMag_3pi,yKronMag_3pi,gKronMagErr_3pi,rKronMagErr_3pi,iKronMagErr_3pi,zKronMagErr_3pi,yKronMagErr_3pi,ps_score_3pi,SN_ra,SN_dec,sn_class,sn_redshift,ZTFID
0,126681757766820931,175.77662082,15.56701789,1,16.0256,15.2881,14.9533,14.7114,14.5005,0.002978,0.001497,0.001272,0.001197,0.002235,0.1110298,175.776542,15.567139,NaN,NaN,NaN
1,148231465149294617,146.51493972,33.52836114,1,18.2387,17.9746,17.875,17.8532,17.7821,0.004157,0.004871,0.004281,0.014407,0.019814,0.021125,146.514792,33.528250,SN II,0.03800,ZTF18aacemcn
2,153731349546771375,134.95471695,38.10897029,1,18.7478,19.0585,18.6466,18.8301,18.6221,0.006392,0.009475,0.004919,0.017235,0.038667,0.01741071,134.954667,38.109056,SN II,0.07247,ZTF18aacnlxz
3,169101206490177628,120.64903126,50.9224606,1,16.5325,15.6522,15.2551,15.0064,14.7582,0.002335,0.002128,0.000867,0.001623,0.003512,0.07590476,120.648958,50.922528,SN Ia,0.05295,ZTF18aadlaxo
4,159661026075384557,102.60753619,43.05326583,1,nan,14.2309,13.166,12.9816,12.8047,nan,0.000836,0.000408,0.001003,0.001687,0.1546488,102.607542,43.053222,SN IIn,0.01885,ZTF18aadmssd


In [3]:
def get_angular_separation(ra1, dec1, ra2, dec2, unit1='deg', unit2='deg'):
    """Function to calculate angular separation in arcseconds"""
    coord1 = SkyCoord(ra1, dec1, unit=unit1)
    coord2 = SkyCoord(ra2, dec2, unit=unit2)
    return coord1.separation(coord2).arcsec

def ab_mag_to_flux(AB_mag: np.ndarray) -> np.ndarray:
    """Convert AB magnitude to flux in mJy"""
    return np.exp((AB_mag - 8.9) / -2.5) / 1000

ab_magerr_to_ferr = lambda sigma_m, f: np.abs(f * np.log(10) * (sigma_m / 2.5))  # transformation on the error of a magnitude turned into flux

# Add angular separation and grib data
bts_df['angular_separation_arcsec'] = get_angular_separation(bts_df['raStack_3pi'], bts_df['decStack_3pi'], bts_df['SN_ra'], bts_df['SN_dec'], unit1='deg', unit2='deg')
grizy = bts_df[['gKronMag_3pi', 'rKronMag_3pi', 'iKronMag_3pi', 'zKronMag_3pi', 'yKronMag_3pi']].to_numpy().astype(float)
grizy_err = bts_df[['gKronMagErr_3pi', 'rKronMagErr_3pi', 'iKronMagErr_3pi', 'zKronMagErr_3pi', 'yKronMagErr_3pi']].to_numpy().astype(float)
angular_seps = bts_df['angular_separation_arcsec'].to_numpy().astype(float)

# Convert the grizy data to mJy
grizy = ab_mag_to_flux(grizy)
grizy_err = ab_magerr_to_ferr(grizy_err, grizy)

grizy, angular_seps

(array([[5.78304399e-05, 7.76735882e-05, 8.88043184e-05, 9.78264782e-05,
         1.06437215e-04],
        [2.38617147e-05, 2.65204273e-05, 2.75983304e-05, 2.78400402e-05,
         2.86431774e-05],
        [1.94653367e-05, 1.71904720e-05, 2.02694592e-05, 1.88349706e-05,
         2.04690764e-05],
        ...,
        [8.85914435e-05, 1.09407044e-04, 1.20300801e-04, 1.30675255e-04,
         1.40436479e-04],
        [5.07116303e-05, 2.32794311e-04, 2.47516013e-04, 1.47176902e-04,
         1.98009553e-04],
        [5.57476512e-05, 8.05401066e-05, 9.51519682e-05, 1.10263756e-04,
         1.00757363e-04]]),
 array([ 0.51487243,  0.59790783,  0.33839217, ...,  0.80115689,
        26.58843375,  6.92645833]))

In [4]:
nan_mask = ~(np.sum(np.isnan(grizy), axis=1)==0)

# Use the SPLASH pipeline

In [5]:
import importlib
import SPLASH.pipeline
importlib.reload(SPLASH.pipeline)
from SPLASH.pipeline import Splash_Pipeline

# Load pipeline object
pipeline = Splash_Pipeline(pipeline_version='weighted_full_band',   # the default version of the pipeline
                           pre_transformed=False,                   # whether the given data is pre-logged and nnormalized
                           within_4sigma=True,                      # whether we only want to classify objects with properties within 4-sigma of the training set
                           nan_thresh_ratio=1.0)                    # to keep this notebook concise, we are allowing the pipeline to imput any num of nans

/Users/adamboesky/opt/anaconda3/envs/ay98_updated/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNNImputer from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
# Predict the classes. n_resamples is the number of boostraps for getting the median predicted host properties.
classes = pipeline.predict_classes(grizy, angular_seps, grizy_err, n_resamples=50)

In [10]:
print(f'Number of each class:\n{pd.Series(classes).value_counts()}\n\nThe class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma.')

Number of each class:
 0    4157
 4     272
-1      52
 2      15
 1       1
Name: count, dtype: int64

The class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma.


In [9]:
# We can also take a look at the predicted host properties of each supernova
props, props_err = pipeline.predict_host_properties(grizy, grizy_err, n_resamples=50, return_normalized=False)  # return_normalized will return the properties normalized with the train mean and std
props, props_err  # in order (log(mass), log(sfr), redshift)

(array([[ 7.59569646, -1.24039593,  0.49556274],
        [ 7.6741121 , -0.69263562,  0.5169176 ],
        [ 7.6866259 , -0.61500514,  0.52030485],
        ...,
        [ 7.61913245, -1.46257195,  0.47561322],
        [ 7.94424799, -0.88291387,  0.64351562],
        [ 7.63948392, -1.21004902,  0.502703  ]]),
 array([[0.00209612, 0.00267932, 0.00095847],
        [0.00232109, 0.00441687, 0.00062875],
        [0.00277292, 0.00303228, 0.00129271],
        ...,
        [0.00131126, 0.00178787, 0.00053433],
        [0.00164465, 0.001824  , 0.00074925],
        [0.00224556, 0.00326122, 0.0010657 ]]))